# Problem 36: Double-base palindromes

The decimal number, $585 = 1001001001_2$ (binary), is palindromic in both bases.

Find the sum of all numbers, less than $\texttt{n}$, whereas 1000 <= $\texttt{n}$ <= 1000000, which are palindromic in base 10 and base 2.

(Please note that the palindromic number, in either base, may not include leading zeros.)

In [1]:
def sequences(seq_len, elements):
    # generator of all sequences length seq_len consisting only of
    # items in elements (will not be unique if items not unique)
    if seq_len == 0 or len(elements) <= 1:
        yield elements[0:1] * seq_len
    else:
        for el in elements:
            for seq in sequences(seq_len-1, elements):
                yield el + seq

In [2]:
def palindromes(pal_len, elements):
    # generator of all palindrome sequences length pal_len consisting
    # only of items in elements (will not be unique if items not unique)
    for seq in sequences((pal_len+1)//2, elements):
        yield seq[pal_len%2:][::-1] + seq

In [3]:
def bodyPalindromes(pal_len, body_elements, univ_elements):
    # generator of all palindrome sequences length pal_len consisting
    # only of items in body_elements+univ_elements, with no items
    # in body elements at the ends (will not be unique if items not unique)
    if pal_len <= 1:
        for pal in palindromes(pal_len, univ_elements):
            yield pal
    else:
        for i in range(len(univ_elements)):
            el = univ_elements[i:i+1]
            for pal in palindromes(pal_len-2, body_elements+univ_elements):
                yield el + pal + el

In [4]:
def doubleBasePalindromes(n):
    # sum of all numbers below n which are palindromic in both
    # base 10 and base 2
    body_digits = '02468'
    univ_digits = '13579'
    max_len = len(str(n-1))
    tot = 0
    for pal_len in range(1,max_len+1):
        for dec in bodyPalindromes(pal_len, body_digits, univ_digits):
            if int(dec) >= n:
                continue
            binary = bin(int(dec))[2:]
            is_pal = True
            for i in range(len(binary)//2):
                if binary[i] != binary[-i-1]:
                    is_pal = False
                    break
            tot += int(dec) if is_pal else 0
    return tot

In [5]:
doubleBasePalindromes(1000000)
# should return 872187

872187

# Problem 37: Truncatable primes
The number 3797 has an interesting property. Being prime itself, it is possible to continuously remove digits from left to right, and remain prime at each stage: 3797, 797, 97, and 7. Similarly we can work from right to left: 3797, 379, 37, and 3.

Find the sum of the only eleven primes that are both truncatable from left to right and right to left.

NOTE: 2, 3, 5, and 7 are not considered to be truncatable primes.

In [6]:
def greatestCommonDivisor(a, b):
    # greatest common divisor of a and b
    r = a % b
    while r:
        a = b
        b = r
        r = a % b
    return b

In [7]:
def isPrime(n):
    # True if n is prime, else False
    if n < 2: return False
    root = int(n**0.5)
    for i in range(2, root+1):
        if n % i == 0:
            return False
    return True

In [8]:
def truncatablePrimes():
    # sum of all integers which are both left- and right-truncatable
    # primes in base 10
    base = 10
    single_digit_primes = []
    addons = []
    for i in range(1, base):
        if greatestCommonDivisor(i, base) == 1:
            addons += [i]
        if isPrime(i):
            single_digit_primes += [i]
    tot = 0
    r_trunc_primes = []
    l_trunc_primes = []
    for p in single_digit_primes:
        for a in addons:
            rn = p * base + a
            if isPrime(rn):
                r_trunc_primes += [rn]
                if isPrime(a):
                    tot += rn
            ln = a * base + p
            if isPrime(ln):
                l_trunc_primes += [ln]
    power = 2
    while l_trunc_primes != [] or r_trunc_primes != []:
        new_r_trunc_primes = []
        new_l_trunc_primes = []
        for a in addons:
            for r_prime in r_trunc_primes:
                rn = base * r_prime + a
                if isPrime(rn):
                    new_r_trunc_primes += [rn]
                    if rn % (base**power) in l_trunc_primes:
                        tot += rn
            for l_prime in l_trunc_primes:
                ln = a * (base**power) + l_prime
                if isPrime(ln):
                    new_l_trunc_primes += [ln]
        r_trunc_primes = new_r_trunc_primes
        l_trunc_primes = new_l_trunc_primes
        power += 1
    return tot

In [9]:
truncatablePrimes()
# should return 748317

748317

# Problem 38: Pandigital multiples

Take the number 192 and multiply it by each of 1, 2, and 3:

192 × 1 = 192
192 × 2 = 384
192 × 3 = 576
By concatenating each product we get the 1 to 9 pandigital, 192384576. We will call 192384576 the concatenated product of 192 and (1,2,3)

The same can be achieved by starting with 9 and multiplying by 1, 2, 3, 4, and 5, giving the pandigital, 918273645, which is the concatenated product of 9 and (1,2,3,4,5).

What is the largest 1 to 9 pandigital 9-digit number that can be formed as the concatenated product of an integer with (1,2, ... , n) where n > 1?

In [10]:
import string

In [11]:
def descendingSubsets(elements, max_size, already_sorted=False, trunc_switch=False):
    # generator of subsets of elements of maximum size max_size
    # in descending order (if trunc_switch is set to True, 
    # subsets which are right-truncated versions of another subset
    # are considered to be larger than that subset)
    if max_size == 0:
        return
    else:
        if not already_sorted:
            if isinstance(elements, str):
                elements = ''.join(sorted(elements))
            else:
                elements = sorted(elements)
        for i in reversed(range(len(elements))):
            el = elements[i:i+1]
            without_el = elements[:i] + elements[i+1:]
            if trunc_switch:
                yield el
            for perm in descendingSubsets(without_el, max_size-1, True, trunc_switch):
                yield el + perm
            if not trunc_switch:
                yield el

In [12]:
all_digits = string.digits + string.ascii_lowercase

In [13]:
def toInt(num_str, base):
    # int value of num_str (when given in base base using first
    # base characters from all_digits)
    if base > len(all_digits):
        print('Base too large, must be <= %s' % (len(all_digits)))
        return
    digits = all_digits[:base]
    num_int = 0
    for i, d in enumerate(reversed(num_str)):
        num_int += digits.index(d) * base**i
    return num_int

In [14]:
def toStr(num_int, base):
    # string value of integer num_int converted to base base using
    # first base characters of all_digits
    if base > len(all_digits):
        print('Base too large, must be <= %s' % (len(all_digits)))
        return
    digits = all_digits[:base]
    num_str = ''
    while num_int:
        d = num_int % base
        num_str = digits[d] + num_str
        num_int = (num_int - d) // base
    return num_str

In [15]:
def pandigitalMultiples(base):
    # highest pandigital number (in base base) which can be formed
    # by concatenating the products of an integer with (1,2,...,n)
    # where n > 1
    if base > len(all_digits):
        print('Base too large, must be <= %s' % (len(all_digits)))
    digits = all_digits[1:base]
    max_digits = (base - 1) // 2
    max_pandigital = ''
    for start_num in descendingSubsets(digits, max_digits, trunc_switch=True):
        if start_num < max_pandigital[:len(start_num)]:
            break
        rem_digits = digits
        for d in start_num:
            rem_digits = rem_digits.replace(d, '')
        mult = 2
        potential_pandigital = True
        while rem_digits and potential_pandigital:
            product = mult * toInt(start_num, base)
            for d in toStr(product, base):
                if d in rem_digits:
                    rem_digits = rem_digits.replace(d, '')
                else:
                    potential_pandigital = False
                    break
            mult += 1
        if potential_pandigital:
            pandigital = start_num
            for m in range(2, mult):
                pandigital += toStr(m * toInt(start_num, base), base)
            if pandigital > max_pandigital:
                max_pandigital = pandigital
    else:
        print('No pandigital numbers found')
        return
    return max_pandigital

In [16]:
pandigitalMultiples(10)
# should return '932718654'

'932718654'

# Problem 39: Integer right triangles

If $p$ is the perimeter of a right angle triangle with integral length sides, $(a,b,c)$, there are exactly three solutions for $p=120$.

$(20,48,52), (24,45,51), (30,40,50)$

For which value of $p$ less than or equal to $N$ is the number of solutions maximised?

In [17]:
import numpy as np

In [18]:
def intRightTriangles(N):
    # right angled triangle perimeter less than N which has the
    # largest number of integer side length solutions
    num_triangles = np.zeros(N+1, int)
    max_m = int(((1 + 4*N)**0.5 + 1) / 4)
    for m in range(2, max_m + 1):
        for n in range(1 + m%2, m, 2):
            if np.gcd(m, n) == 1:
                p = 2*(m**2 + m*n)
                num_triangles[p::p] += 1
    return np.argmax(num_triangles)

In [19]:
intRightTriangles(1000)
# should return 840

840

# Problem 40: Champernowne's constant

An irrational decimal fraction is created by concatenating the positive integers:

$0.123456789101112131415161718192021...$

It can be seen that the 12th digit of the fractional part is 1.

If $d_n$ represents the $n$th digit of the fractional part, find the value of the following expression.

$d_1 × d_{10} × d_{100} × d_{1000} × d_{10000} × d_{100000} × d_{1000000}$

In [20]:
def chapernowneDigit(N):
    # Nth significat digit of Champernowne's constant
    N -= 1
    base = 10
    power = 0
    digit_count = base - 1
    while digit_count <= N:
        N -= digit_count
        power += 1
        digit_count = (power + 1) * (base - 1) * base**power
    q, r = divmod(N, power + 1)
    num = base**power + q
    digit = str(num)[r]
    return int(digit)

In [21]:
def champernownesConstant(max_power):
    # product of all digits at positions 10^n (n = 1,2,...,max_power)
    # in Champernowne's constant
    base = 10
    tot = 1
    for power in range(max_power):
        tot *= chapernowneDigit(base**power)
    return tot

In [22]:
champernownesConstant(6)
# should return 210

210